<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/section_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip uninstall gensim -y

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Successfully uninstalled gensim-4.1.2


In [71]:
!pip install gensim

  Using cached gensim-4.1.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)


In [72]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [73]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [74]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Download Data

### Basic Chords (major triads, minor triads, dom7)

In [75]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO' -O data.csv

--2021-11-17 16:23:35--  https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO
Resolving docs.google.com (docs.google.com)... 142.251.6.139, 142.251.6.100, 142.251.6.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.6.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7is0sqnmjp9f8ak8l307n8g176b4e0hb/1637166150000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download [following]
--2021-11-17 16:23:36--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7is0sqnmjp9f8ak8l307n8g176b4e0hb/1637166150000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 108.177.120.132, 2607:f8b0:4001:c18::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs

### Simplified Chords

In [76]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb' -O data.csv

--2021-11-17 16:23:37--  https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb
Resolving docs.google.com (docs.google.com)... 74.125.70.113, 74.125.70.138, 74.125.70.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.70.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5ffgr49cgj65ngdhjhdlgmd2n4qjo3ke/1637166150000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download [following]
--2021-11-17 16:23:38--  https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5ffgr49cgj65ngdhjhdlgmd2n4qjo3ke/1637166150000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download
Resolving doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-docs.googleusercontent.com)... 108.177.120.132, 2607:f8b0:4001:c18::84
Connecting to doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-docs

In [77]:
path_to_file = '/content/data.csv'
df = pd.read_csv(path_to_file, sep='\t', index_col="id")
df = df.reset_index()

In [78]:
df

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 G#7 C#M7 E7 ...
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm7 C7 Bm7 E7 AM7 C7 FM7 Bbm7 Eb7 G#M7 Dm7 G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am7 Am7 Cm7 Cm7 EbM7 EbM7 Em7b5 A7 Dm7 Dm7 Bm7...
...,...,...,...,...,...,...,...,...
5398,5398,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,1,Dm7 G7 Em7b5 A7 Dm7 G#7 G7 C C
5399,5399,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,2,Dm7 G7 Em7b5 A7 Dm7 G#7 G7 C C A7 A7 A7 A7 Dm7...
5400,5400,dataset/trad/You're Nobody Till Somebody Loves...,You're Nobody Till Somebody Loves You,major,1809,A,1,C E7 A7 A7 Dm G7 C Em7 C G7 Em7 Cdim Dm7 Dm7 D...
5401,5401,dataset/trad/Your Feet's Too Big.xml,Your Feet's Too Big,major,1810,NaN,0,G7 G7 C C G7 G7 C F G# C C G7 G7 C F C C G7 C ...


### Meta Data

In [79]:
titles = df.loc[:, ['id', 'tune_id', 'section_id', 'section_name', 'title', 'tune_mode']]

In [80]:
titles

,id,tune_id,section_id,section_name,title,tune_mode
0,0,0,1,A,26-2,major
1,1,0,2,A,26-2,major
2,2,0,3,B,26-2,major
3,3,0,4,A,26-2,major
4,4,1,0,NaN,500 Miles High,minor
...,...,...,...,...,...,...
5398,5398,1808,1,A,You're Lucky To Me,major
5399,5399,1808,2,A,You're Lucky To Me,major
5400,5400,1809,1,A,You're Nobody Till Somebody Loves You,major
5401,5401,1810,0,NaN,Your Feet's Too Big,major


In [81]:
##TODO Now a tune contains multiple rows

In [82]:
titles_dict = titles.to_dict()
sectionid_to_title = titles_dict['title']
sectionid_to_title[1170]

'Emily'

In [83]:
sectionid_to_title_id = titles_dict['tune_id']
sectionid_to_title_id[5396]

1807

In [84]:
titles_rows = titles.to_dict(orient='records')
titles_rows[0]

sectionid_to_section = []
for i, row in enumerate(titles_rows):
  name = f"{row['title']}, section{row['section_id']} ({row['section_name']})"
  sectionid_to_section.append(name)

sectionid_to_section[3]

'26-2, section4 (A)'

In [85]:
title_to_sectionid = {}

for row in titles.iterrows():
  title = row[1]['title']
  if title not in title_to_sectionid:
    title_to_sectionid[title] = [row[1]['id']]
  else:
    title_to_sectionid[title].append(row[1]['id'])


In [86]:
title_to_sectionid

{'26-2': [0, 1, 2, 3],
 '500 Miles High': [4],
 '502 Blues': [5],
 '52nd Street Theme': [6, 7, 8, 9],
 '9.20 Special': [10, 11, 12, 13],
 'A Ballad': [14, 15, 16, 17],
 'A Beautiful Friendship': [18, 19, 20, 21],
 'A Blossom Fell': [22, 23],
 'A Certain Smile': [24, 25, 26, 27],
 'A Child Is Born': [28, 29],
 'A Felicidade': [30, 31, 32, 33, 34],
 'A Fine Romance': [35, 36, 37, 38],
 'A Flower Is A Lovesome Thing': [39, 40, 41, 42],
 'A Foggy Day': [43, 44, 45, 46],
 'A Ghost Of A Chance': [47, 48, 49, 50],
 'A Kiss To Build A Dream On': [51, 52, 53, 54],
 'A Little Tear': [55, 56],
 'A Lovely Way To Spend An Evening': [57, 58, 59, 60],
 'A Night In Tunisia': [61, 62, 63, 64],
 'A Nightingale Sang In Berkeley Square': [65, 66, 67, 68],
 'A Pretty Girl Is Like A Melody': [69, 70],
 "A Sleepin' Bee": [71, 72, 73, 74],
 'A Smooth One': [75, 76, 77, 78],
 'A Sound For Sore Ears': [79, 80, 81],
 'A Sunday Kind Of Love': [82, 83, 84, 85],
 'A Taste Of Honey': [86, 87],
 'A Time For Love': [8

# List Tunes for Test Data

In [87]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_sectionid[reference], title_to_sectionid[compare]])
  

In [88]:
contrafacts_test[:5]

[[[0, 1, 2, 3], [810, 811, 812, 813]],
 [[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[104, 105, 106, 107], [209, 210, 211, 212]],
 [[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]]]

## Helpers for N-Grams

In [89]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [90]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

# TF-IDF

## Build Model

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [211]:
lines = df.loc[:, 'chords'].tolist()
data = [line.split(' ') for line in lines]

In [237]:
for i in data[:10]:
  print(i)

['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7']
['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7']
['Gm7', 'C7', 'Bm7', 'E7', 'AM7', 'C7', 'FM7', 'Bbm7', 'Eb7', 'G#M7', 'Dm7', 'G7']
['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7']
['Am7', 'Am7', 'Cm7', 'Cm7', 'EbM7', 'EbM7', 'Em7b5', 'A7', 'Dm7', 'Dm7', 'Bm7b5', 'Bm7b5', 'Bbm7', 'Bbm7', 'Fm7', 'Fm7', 'E7', 'E7', 'Fm7', 'Fm7', 'C#M7', 'C#M7', 'Fm7', 'Fm7', 'C#M7', 'C#M7']
['Am7', 'C#M7', 'Bm7b5', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Cm7', 'F7', 'BbM7', 'G#m7', 'C#7', 'F#m7b5', 'B7', 'EM7', 'EM7', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Am7', 'C#M7', 'Bm7b5', 'E7', 'Cm7', 'F7', 'BbM7', 'G#m7', 'C#7', 'F#m7b5', 'B7', 'Em7', 'Em7']
['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'G7', 'C']
['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', '

In [220]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  print(data_ngrams[0])
  data_tfidf = [" ".join(i) for i in data_ngrams]

  # make sure that no processing on the chord symbols is done!
  tv = TfidfVectorizer(min_df = 0., 
                     max_df = 1., 
                     norm = 'l2', 
                     use_idf = True,
                     stop_words=None,
                     lowercase=False,
                     token_pattern='\\b[\\w#]+')

  tfidf_vectorizer[n] = tv_
  tfidf_model[n] = tv.fit_transform(data_tfidf)

['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7']
['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7']
['CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7']


In [ ]:
for i in range(len(tfidf_vectorizer)):
  print(tfidf_vectorizer[i].get_feature_names_out())

In [221]:
i = 1

tv_matrix = tfidf_model[i].toarray()

vocab = tv[i].get_feature_names_out()
tfidf_chords = pd.DataFrame(tv_matrix, columns = vocab)
tfidf_chords

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,A,A13,A6,A7,AM7,Adim,Adim7,Am,Am6,Am7,Am7b5,AmM7,B,B13,B6,B7,BM7,Bb,Bb13,Bb6,Bb7,BbM7,Bbdim,Bbdim7,Bbm,Bbm6,Bbm7,Bbm7b5,BbmM7,Bdim,Bdim7,Bm,Bm6,Bm7,Bm7b5,BmM7,C,C#,C#13,C#6,...,F#dim7,F#m,F#m6,F#m7,F#m7b5,F#mM7,F13,F6,F7,FM7,Fdim,Fdim7,Fm,Fm6,Fm7,Fm7b5,FmM7,G,G#,G#13,G#6,G#7,G#M7,G#dim,G#dim7,G#m,G#m7,G#m7b5,G13,G6,G7,GM7,Gdim,Gdim7,Gm,Gm6,Gm7,Gm7b5,GmM7,b5
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.183958,0.0,0.0,0.0,0.0,0.0,0.244503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.219451,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.270663,0.355014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218613,0.0,0.0,0.0,0.0,0.0,0.247727,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.339563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.152386,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.493039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151803,0.0,0.0,0.0,0.0,0.0,0.172020,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.425097,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388876,0.0,0.0,0.0,0.0,0.0,0.0,0.354405,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.222138,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.359360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.110645,0.0,0.0,0.0,0.0,0.0,0.250760,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.339563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.152386,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.493039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.151803,0.0,0.0,0.0,0.0,0.0,0.172020,0.0,0.0,0.0
4,0.0,0.0,0.0,0.054763,0.000000,0.0,0.0,0.000000,0.0,0.132782,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277299,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.199106,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.611458,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,0.0,0.0,0.0,0.215152,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.612088,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.383773,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309971,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5399,0.0,0.0,0.0,0.513191,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.486661,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.305132,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.369680,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5400,0.0,0.0,0.0,0.316703,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.525578,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190115,0.0,0.0,0.0,0.0

The `tfidf_model` is a list of the 4 models fitted with ngram size 1...4. 
Each of the model contains a sparse matrix with tunes being in the rows and the vocabulary being in the columns.

In [224]:
tfidf_model[1][5000]

<1x143 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [225]:
tfidf_model

{1: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43611 stored elements in Compressed Sparse Row format>,
 2: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43611 stored elements in Compressed Sparse Row format>,
 3: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43606 stored elements in Compressed Sparse Row format>,
 4: <5403x143 sparse matrix of type '<class 'numpy.float64'>'
 	with 43603 stored elements in Compressed Sparse Row format>}

## Test for single tunes

In [226]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

Evaluate TF-IDF Model for various ngram sizes.

In [227]:
model1 = 1
model2 = 2
model3 = 3
model4 = 4

df_sim = pd.DataFrame(columns=['reference_title',
                               'similar_title',
                               'section1', 
                               'section2', 
                               f'sim{model1}', 
                               f'sim{model2}',
                               f'sim{model3}',
                               f'sim{model4}',
                               ])

for tune in tunes_eval_list:
  print(tune)
  for s1 in title_to_sectionid[tune]:
      print(s1)
      for s2 in range(len(data_tfidf)):
        if s2 not in title_to_sectionid[tune]:
          sim1 = cosine_similarity(tfidf_model[model1][s1], tfidf_model[model1][s2])[0][0]
          sim2 = cosine_similarity(tfidf_model[model2][s1], tfidf_model[model2][s2])[0][0]
          sim3 = cosine_similarity(tfidf_model[model3][s1], tfidf_model[model3][s2])[0][0]
          sim4 = cosine_similarity(tfidf_model[model4][s1], tfidf_model[model4][s2])[0][0]
          if sim1 > 0.8:
            df_sim.loc[len(df_sim)] = [tune,
                                      sectionid_to_title[s2],
                                      sectionid_to_section[s1], 
                                      sectionid_to_section[s2], 
                                      sim1, 
                                      sim2,
                                      sim3,
                                      sim4,
                                      ]

df_sim = df_sim.sort_values(f'sim{model2}', ascending=False)

Sweet Sue, Just You
3752
3753
3754
3755
These Foolish Things
3866
3867
3868
3869
Blue Moon
496
497
498
499
All Of Me
198
199
200
201
All God's Chillun Got Rhythm
188
189
190
191
I Got Rhythm
1647
1648
1649
1650
4821
4822
4823
4824
Bye Bye Blackbird
653
654
4569
4570
4571
Old Fashioned Love
5056
Dinah
997
998
999
1000
Honeysuckle Rose
1538
1539
1540
1541
4793
4794
4795
4796
Misty
2676
2677
2678
2679


In [228]:
from google.colab import files

file = 'tfidf_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [229]:
df_sim.head(50)

,reference_title,similar_title,section1,section2,sim1,sim2,sim3,sim4
926,I Got Rhythm,Perdido,"I Got Rhythm, section3 (B)","Perdido, section3 (B)",1.000000,1.000000,1.000000,1.000000
927,I Got Rhythm,Rhythm-a-ning,"I Got Rhythm, section3 (B)","Rhythm-a-ning, section3 (B)",1.000000,1.000000,1.000000,1.000000
806,I Got Rhythm,I Love Being Here With You,"I Got Rhythm, section3 (B)","I Love Being Here With You, section3 (B)",1.000000,1.000000,1.000000,1.000000
936,I Got Rhythm,Two Not One,"I Got Rhythm, section3 (B)","Two Not One, section3 (B)",1.000000,1.000000,1.000000,1.000000
937,I Got Rhythm,Ain't That A Grand And Glorious Feeling,"I Got Rhythm, section3 (B)","Ain't That A Grand And Glorious Feeling, secti...",1.000000,1.000000,1.000000,1.000000
795,I Got Rhythm,Anthropology,"I Got Rhythm, section3 (B)","Anthropology, section3 (B)",1.000000,1.000000,1.000000,1.000000
802,I Got Rhythm,Flintstones,"I Got Rhythm, section3 (B)","Flintstones, section3 (B)",1.000000,1.000000,1.000000,1.000000
800,I Got Rhythm,Dexterity,"I Got Rhythm, section3 (B)","Dexterity, section3 (B)",1.000000,1.000000,1.000000,1.000000
941,I Got Rhythm,Sunday,"I Got Rhythm, section3 (B)","Sunday, section3 (B)",1.000000,1.000000,1.000000,1.000000
943,I Got Rhythm,When I'm Cleaning Windows,"I Got Rhythm, section3 (B)","When I'm Cleaning Windows, section2 (B)",1.000000,1.000000,1.000000,1.000000


## Test of the Contrafacts tunes

In [230]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)


In [231]:
for tune1, tune2 in contrafacts_test:
  print(f"{tune1}, {tune2}")

[0, 1, 2, 3], [810, 811, 812, 813]
[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[104, 105, 106, 107], [209, 210, 211, 212]
[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]
[594], [3746, 3747]
[668, 669, 670, 671], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[707, 708, 709, 710, 711, 712, 713, 714], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[835, 836, 837, 838], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[841], [4028]
[980, 981, 982, 983], [2968, 2969, 2970, 2971]
[984, 985, 986, 987], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[995, 996], [3746, 3747]
[1084, 1085], [1996, 1997]
[1042, 1043, 1044, 1045], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1191, 1192, 1193, 1194], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1216, 1217, 1218, 1219], [2210, 2211, 2212, 2213]
[1271, 1272, 1273, 1274], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[1321], [3722, 5222]
[13

In [232]:
# calculate a dataframe with the cos sim value for the test tunes
df_tfidf = pd.DataFrame(columns=['tune1', 'tune2', 'ngram_size', 'section1', 'section2', 'similarity'])
for ngram_size, tfidf_m in tfidf_model.items():

    for tune1, tune2 in contrafacts_test:
      #print(f"tune1: {tune1}, tune2: {tune2}")
      tune_sim = []
      # loop over all combinations of sections
      for s1 in tune1:
          for s2 in tune2:
                #print(f"ngram: {ngram_size}: {sectionid_to_section[s1]}, {sectionid_to_section[s2]}:  {cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]}")
                row = [sectionid_to_title_id[tune1[0]], sectionid_to_title_id[tune2[0]], ngram_size, sectionid_to_section[s1], sectionid_to_section[s2], cosine_similarity(tfidf_m[s1], tfidf_m[s2])[0][0]]
                df_tfidf.loc[len(df_tfidf)] = row
      


In [233]:
df_tfidf.query('ngram_size == 1').sort_values(['tune1', 'similarity'], ascending=[True, False]).head(50)

,tune1,tune2,ngram_size,section1,section2,similarity
10,0,251,1,"26-2, section3 (B)","Confirmation, section3 (B)",0.691861
6,0,251,1,"26-2, section2 (A)","Confirmation, section3 (B)",0.502416
14,0,251,1,"26-2, section4 (A)","Confirmation, section3 (B)",0.502416
2,0,251,1,"26-2, section1 (A)","Confirmation, section3 (B)",0.498167
3,0,251,1,"26-2, section1 (A)","Confirmation, section4 (A)",0.417841
1,0,251,1,"26-2, section1 (A)","Confirmation, section2 (A)",0.398857
0,0,251,1,"26-2, section1 (A)","Confirmation, section1 (A)",0.382584
11,0,251,1,"26-2, section3 (B)","Confirmation, section4 (A)",0.277554
9,0,251,1,"26-2, section3 (B)","Confirmation, section2 (A)",0.270191
8,0,251,1,"26-2, section3 (B)","Confirmation, section1 (A)",0.248027


In [234]:
df_tfidf_pivot = pd.pivot_table(df_tfidf, index = ['tune1', 'tune2', 'section1', 'section2'], columns = 'ngram_size', values = 'similarity')
df_tfidf_pivot.head(50)

ngram_size                                                                               1  ...         4
tune1 tune2 section1                        section2                                        ...          
0     251   26-2, section1 (A)              Confirmation, section1 (A)            0.382584  ...  0.372820
                                            Confirmation, section2 (A)            0.398857  ...  0.379749
                                            Confirmation, section3 (B)            0.498167  ...  0.390971
                                            Confirmation, section4 (A)            0.417841  ...  0.387522
            26-2, section2 (A)              Confirmation, section1 (A)            0.166310  ...  0.142539
                                            Confirmation, section2 (A)            0.216111  ...  0.155761
                                            Confirmation, section3 (B)            0.502416  ...  0.500740
                                            Confirmation, section4 (A)            0.231294  ...  0.166976
            26-2, section3 (B)              Confirmation, section1 (A)            0.248027  ...  0.185885
                                            Confirmation, section2 (A)            0.270191  ...  0.203248
                                            Confirmation, section3 (B)            0.691861  ...  0.726261
                                            Confirmation, section4 (A)            0.277554  ...  0.203180
            26-2, section4 (A)              Confirmation, section1 (A)            0.166310  ...  0.142539
                                            Confirmation, section2 (A)            0.216111  ...  0.155761
                                            Confirmation, section3 (B)            0.502416  ...  0.500740
                                            Confirmation, section4 (A)            0.231294  ...  0.166976
3     505   52nd Street Theme, section1 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
            52nd Street Theme, section2 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
            52nd Street Theme, section3 (B) I Got Rhythm, section1 (A)            0.240516  ...  0.127823
                                            I Got Rhythm, section2 (A)            0.223873  ...  0.129214
                                            I Got Rhythm, section3 (B)            0.294358  ...  0.318568
                                            I Got Rhythm, section4 (A)            0.186662  ...  0.101162
            52nd Street Theme, section4 (A) I Got Rhythm, section1 (A)            0.346735  ...  0.397223
                                            I Got Rhythm, section2 (A)            0.325613  ...  0.399664
                                            I Got Rhythm, section3 (B)            0.112202  ...  0.073072
                                            I Got Rhythm, section4 (A)            0.292092  ...  0.345881
30    63    Ablution, section1 (A)          All The Things You Are, section1 (A)  0.843729  ...  0.527422
                                            All The Things You Are, section2 (B)  0.052460  ...  0.076243
                                            All The Things You Are, section3 (C)  0.000000  ...  0.000000
                                            All The Things You Are, section4 (

In [235]:
df_tfidf_pivot.query('tune1 == 1170')

ngram_size                                                                     1  ...         4
tune1 tune2 section1                           section2                           ...          
1170  159   These Foolish Things, section1 (A) Blue Moon, section1 (A)  0.782701  ...  0.680510
                                               Blue Moon, section2 (A)  0.670174  ...  0.615911
                                               Blue Moon, section3 (B)  0.419513  ...  0.297998
                                               Blue Moon, section4 (A)  0.782701  ...  0.680510
            These Foolish Things, section2 (A) Blue Moon, section1 (A)  0.675141  ...  0.648572
                                               Blue Moon, section2 (A)  0.586182  ...  0.593161
                                               Blue Moon, section3 (B)  0.426671  ...  0.370118
                                               Blue Moon, section4 (A)  0.675141  ...  0.648572
            These Foolish Things, section3 (B) Blue Moon, section1 (A)  0.232981  ...  0.226942
                                               Blue Moon, section2 (A)  0.200085  ...  0.201910
                                               Blue Moon, section3 (B)  0.189414  ...  0.151101
                                               Blue Moon, section4 (A)  0.232981  ...  0.226942
            These Foolish Things, section4 (A) Blue Moon, section1 (A)  0.774064  ...  0.680519
                                               Blue Moon, section2 (A)  0.659705  ...  0.620852
                                               Blue Moon, section3 (B)  0.516911  ...  0.391904
                                               Blue Moon, section4 (A)  0.774064  ...  0.680519

[16 rows x 4 columns]

In [236]:
df_tfidf_pivot.query('tune1 == 1138')

ngram_size                                                                            1  ...         4
tune1 tune2 section1                          section2                                   ...          
1138  206   Sweet Sue, Just You, section1 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122
            Sweet Sue, Just You, section2 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122
            Sweet Sue, Just You, section3 (B) Bye Bye Blackbird, section1 (A)  0.165758  ...  0.229472
                                              Bye Bye Blackbird, section2 (B)  0.287453  ...  0.376501
                                              Bye Bye Blackbird, section3 (C)  0.317302  ...  0.642809
            Sweet Sue, Just You, section4 (A) Bye Bye Blackbird, section1 (A)  0.342375  ...  0.260481
                                              Bye Bye Blackbird, section2 (B)  0.146986  ...  0.128608
                                              Bye Bye Blackbird, section3 (C)  0.064185  ...  0.103122

[12 rows x 4 columns]

# SVD / LSI Example

In [298]:
data_ngrams = list_to_ngrams(data, n=4)

for line in data_ngrams[:10]:
  print(line)

['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-Eb7', 'C7-FM7-Eb7-G#M7', 'FM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-CM7']
['Gm7-C7-Bm7-E7', 'C7-Bm7-E7-AM7', 'Bm7-E7-AM7-C7', 'E7-AM7-C7-FM7', 'AM7-C7-FM7-Bbm7', 'C7-FM7-Bbm7-Eb7', 'FM7-Bbm7-Eb7-G#M7', 'Bbm7-Eb7-G#M7-Dm7', 'Eb7-G#M7-Dm7-G7']
['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-Eb7', 'C7-FM7-Eb7-G#M7', 'FM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-CM7']
['Am7-Am7-Cm7-Cm7', 'Am7-Cm7-Cm7-EbM7', 'Cm7-Cm7-EbM7-EbM7', 'Cm7-EbM7-EbM7-Em7b5', 'EbM7-EbM7-Em7b5-A7', 'EbM7-Em7b5-A7-Dm7', 'Em7b5-A7-Dm7-Dm7', 'A7-Dm7-Dm7-Bm

In [300]:
from gensim import corpora, models, similarities
from collections import defaultdict

frequency = defaultdict(int)
for text in data_ngrams:
    for token in text:
        frequency[token] += 1

data_ngrams = [[token for token in text if frequency[token] > 1] for text in data_ngrams]
dictionary = corpora.Dictionary(data_ngrams)

# doc2bow counts the number of occurences of each distinct word,
# converts the word to its integer word id and returns the result
# as a sparse vector

corpus = [dictionary.doc2bow(text) for text in data_ngrams]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)  # num_topics can be maximum the size of the number of unique tokens

2021-11-17 22:10:50,945 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-17 22:10:51,069 : INFO : built Dictionary(8838 unique tokens: ['Am7-D7-Dm7-G7', 'B7-EM7-G7-Gm7', 'CM7-Eb7-G#M7-B7', 'E7-Am7-D7-Dm7', 'EM7-G7-Gm7-C7']...) from 5403 documents (total 54912 corpus positions)
2021-11-17 22:10:51,071 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(8838 unique tokens: ['Am7-D7-Dm7-G7', 'B7-EM7-G7-Gm7', 'CM7-Eb7-G#M7-B7', 'E7-Am7-D7-Dm7', 'EM7-G7-Gm7-C7']...) from 5403 documents (total 54912 corpus positions)", 'datetime': '2021-11-17T22:10:51.071646', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-17 22:10:51,135 : INFO : using serial LSI version on this node
2021-11-17 22:10:51,140 : INFO : updating model with new documents
2021-11-17 22:10:51,144 : INFO : preparing a new chunk of documents
2021-11-17 22:10:51,185 : INFO : 

In [301]:
lsi.get_topics().shape

(200, 8838)

In [306]:
title_to_sectionid['These Foolish Things']

[3866, 3867, 3868, 3869]

In [308]:
# add new sample

i = 3868
print(f"Check similarity for {sectionid_to_section[i]}")
tune = data_ngrams[i]
vec_bow = dictionary.doc2bow(tune)

# convert the query to LSI space
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
print(len(vec_lsi))

2021-11-17 22:12:42,221 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


Check similarity for These Foolish Things, section3 (B)


2021-11-17 22:12:42,724 : INFO : creating matrix with 5403 documents and 200 features


200


In [309]:
# perform a similarity query against the corpus
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

print(sims)
print(len(sims))

[(3868, 1.0), (2032, 0.99550563), (825, 0.90867275), (3595, 0.9029014), (1941, 0.8897047), (3881, 0.80056864), (1986, 0.75324214), (1987, 0.75324214), (1989, 0.75324214), (3985, 0.7485474), (4306, 0.7480322), (1867, 0.7464211), (1906, 0.7427673), (1672, 0.7391666), (888, 0.7348897), (2361, 0.73320484), (781, 0.72818387), (1473, 0.7069675), (3032, 0.69117045), (1787, 0.67645764), (1606, 0.67027307), (4267, 0.6640296), (2719, 0.66390413), (2324, 0.6558944), (252, 0.6540999), (1724, 0.6499111), (2350, 0.644735), (1368, 0.6182363), (49, 0.58358204), (3245, 0.5776606), (1153, 0.55178595), (1877, 0.55106956), (991, 0.5484789), (1980, 0.5316942), (1716, 0.51938164), (3903, 0.51702267), (3242, 0.51338756), (1717, 0.51241255), (33, 0.50913405), (2447, 0.50532836), (1878, 0.5046196), (3642, 0.5041847), (102, 0.4941488), (3628, 0.4914485), (3630, 0.4914485), (38, 0.48977363), (3221, 0.48530933), (3757, 0.48487228), (205, 0.47490358), (105, 0.4735892), (189, 0.4718058), (1729, 0.4624539), (3901, 0

In [310]:
print(f"Similarity for '{sectionid_to_section[i]}'")
print()

for id, sim in sims[:15]:
  print(f"{sim:.3f} {sectionid_to_section[id]}")

Similarity for 'These Foolish Things, section3 (B)'

1.000 These Foolish Things, section3 (B)
0.996 Isn't It A Pity, section3 (B)
0.909 Coral, section0 (nan)
0.903 Soultrane, section3 (B)
0.890 Imagination, section3 (B)
0.801 They Say It's Wonderful, section3 (B)
0.753 In Your Own Sweet Way, section1 (A)
0.753 In Your Own Sweet Way, section2 (A)
0.753 In Your Own Sweet Way, section4 (A)
0.749 Too Young To Go Steady, section3 (B)
0.748 Yardbird Suite, section3 (B)
0.746 I've Got My Love To Keep Me Warm, section3 (B)
0.743 If I Had You, section3 (B)
0.739 I Hear A Rhapsody, section3 (B)
0.735 Cynthia's In Love, section3 (B)


#### Test for single tunes

In [312]:
# test a sample

n = 4

for tune in tunes_eval_list:
  print(f"\n\n*** {tune} ***")
  for s1 in title_to_sectionid[tune]:

    print(f"\nCheck similarity for {sectionid_to_section[s1]}")
    tune = data_ngrams[s1]
    vec_bow = dictionary.doc2bow(tune)

    # convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    index = similarities.MatrixSimilarity(lsi[corpus])

    # perform a similarity query against the corpus
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])

    for id, sim in sims[:15]:
      print(f"{sim:.3f} {sectionid_to_section[id]}")

2021-11-17 22:14:06,706 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)




*** Sweet Sue, Just You ***

Check similarity for Sweet Sue, Just You, section1 (A)


2021-11-17 22:14:07,199 : INFO : creating matrix with 5403 documents and 200 features
2021-11-17 22:14:08,239 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Sweet Sue, Just You, section1 (A)
1.000 Sweet Sue, Just You, section2 (A)
1.000 Sweet Sue, Just You, section4 (A)
0.665 A Blossom Fell, section1 (A)
0.655 I Mean You, section5 (A)
0.604 How Are Things In Glocca Morra, section2 (A)
0.588 My Secret Love, section2 (A)
0.577 Alone Too Long, section2 (A)
0.570 Someday (You'll Be Sorry), section2 (B)
0.563 I Want To Be Happy, section2 (A)
0.559 Brazil (Aquarela Do Brasil), section1 (verse)
0.559 Seven Come Eleven, section1 (A)
0.559 Seven Come Eleven, section2 (A)
0.559 Seven Come Eleven, section4 (A)
0.555 Breeze And I, The, section1 (A)

Check similarity for Sweet Sue, Just You, section2 (A)


2021-11-17 22:14:08,749 : INFO : creating matrix with 5403 documents and 200 features
2021-11-17 22:14:09,801 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


...

1.000 Misty, section2 (A)
0.791 Misty, section4 (A)
0.749 My Way, section1 (A)
0.749 My Way, section2 (A)
0.742 I Remember You, section2 (A)
0.741 Easy Street, section3 (B)
0.731 Portrait Of Jennie, section2 (A)
0.704 You're Everything, section1 (A)
0.701 My Secret Love, section3 (B)
0.694 Mahjong, section5 (C)
0.688 Taking A Chance On Love, section3 (B)
0.675 It's Easy To Remember, section3 (B)
0.674 You're Everything, section3 (C)
0.668 Breeze And I, The, section4 (C)
0.645 Do Nothin' Til You Hear From Me, section2 (A)

Check similarity for Misty, section3 (B)


2021-11-17 22:15:22,993 : INFO : creating matrix with 5403 documents and 200 features
2021-11-17 22:15:24,015 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Misty, section3 (B)
0.728 Upper Manhattan Medical Group, section1 (A)
0.726 Upper Manhattan Medical Group, section2 (A)
0.726 Upper Manhattan Medical Group, section4 (A)
0.711 This Can't Be Love, section3 (B)
0.694 S.O.S., section5 (D)
0.694 Laura, section1 (A)
0.694 Laura, section3 (A)
0.691 A Sound For Sore Ears, section3 (C)
0.685 P.S. I Love You, section3 (B)
0.680 Strike Up The Band, section2 (B)
0.665 Só Danço Samba, section3 (B)
0.658 In A Mellow Tone (In A Mellotone), section1 (A)
0.658 In A Mellow Tone (In A Mellotone), section3 (A)
0.648 Moonlight Serenade, section3 (B)

Check similarity for Misty, section4 (A)


2021-11-17 22:15:24,570 : INFO : creating matrix with 5403 documents and 200 features


1.000 Misty, section4 (A)
0.791 Misty, section2 (A)
0.715 Portrait Of Jennie, section4 (A)
0.695 Easy Street, section3 (B)
0.695 Do Nothin' Til You Hear From Me, section4 (A)
0.694 I Remember You, section2 (A)
0.684 My Way, section1 (A)
0.684 My Way, section2 (A)
0.680 Portrait Of Jennie, section2 (A)
0.656 My Secret Love, section3 (B)
0.652 You're Everything, section1 (A)
0.648 Mahjong, section5 (C)
0.643 Taking A Chance On Love, section3 (B)
0.636 I Remember You, section4 (A)
0.631 It's Easy To Remember, section3 (B)


# Doc2Vec Embeddings

## Read and Tag Data

In [105]:
df.head()

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 G#7 C#M7 E7 ...
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm7 C7 Bm7 E7 AM7 C7 FM7 Bbm7 Eb7 G#M7 Dm7 G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am7 Am7 Cm7 Cm7 EbM7 EbM7 Em7b5 A7 Dm7 Dm7 Bm7...


In [106]:
def read_corpus(df, ngram_size, tokens_only=False):
    for i, row in df.iterrows():
      tokens = ngrams(row['chords'].split(), ngram_size, '-')
      if tokens_only:
          yield tokens
      else:
          # for training data, add tags
          yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id'], row['tune_id']])
          #yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id']])

## Doc2Vec Hyperparameters

In [129]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [130]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [131]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  #'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 4, 
                                 #10
                                 ],
                      'ngrams': [
                                 1, 
                                 2, 
                                 3, 
                                 4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=40, 
    min_count=1,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(df, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # calculate the model
                if algo == 'd2v':
                    print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}")
                    print(f"Corpus: {train_corpus[0]}")
                    model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                          dm = dm,
                                                          dm_concat = concat,
                                                          )
                    model.build_vocab(train_corpus)
                    model.train(train_corpus, 
                                total_examples=model.corpus_count, 
                                epochs=model.epochs)

                else:
                    model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                model.save(file_name + '.model')

                file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                with open(file_name + '.train', 'wb') as f:
                      # Pickle the 'data' dictionary using the highest protocol available.
                      pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-17 18:44:52,438 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,t2)', 'datetime': '2021-11-17T18:44:52.438712', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-17 18:44:52,440 : INFO : collecting all words and their counts
2021-11-17 18:44:52,444 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-17 18:44:52,469 : INFO : collected 154 word types and 5403 unique tags from a corpus of 5403 examples and 82790 words
2021-11-17 18:44:52,475 : INFO : Creating a fresh vocabulary
2021-11-17 18:44:52,480 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 154 unique words (100.0%% of original 154, drops 0)', 'datetime': '2021-11-17T18:44:52.480283', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7'], [0, 0])


2021-11-17 18:44:53,075 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:44:53,082 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:44:53,084 : INFO : EPOCH - 1 : training on 82790 raw words (93596 effective words) took 0.6s, 164384 effective words/s
2021-11-17 18:44:53,627 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:44:53,646 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:44:53,651 : INFO : EPOCH - 2 : training on 82790 raw words (93596 effective words) took 0.6s, 168651 effective words/s
2021-11-17 18:44:54,203 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:44:54,212 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:44:54,213 : INFO : EPOCH - 3 : training on 82790 raw words (93596 effective words) took 0.6s, 169237 effective words/s
2021-11-17 18:44:54,762 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7'], [0, 0])


2021-11-17 18:45:16,586 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:45:16,598 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:45:16,599 : INFO : EPOCH - 1 : training on 82790 raw words (93596 effective words) took 0.6s, 156851 effective words/s
2021-11-17 18:45:17,162 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:45:17,179 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:45:17,180 : INFO : EPOCH - 2 : training on 82790 raw words (93596 effective words) took 0.6s, 164168 effective words/s
2021-11-17 18:45:17,791 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-17 18:45:17,801 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-17 18:45:17,802 : INFO : EPOCH - 3 : training on 82790 raw words (93596 effective words) took 0.6s, 153271 effective words/s
2021-11-17 18:45:18,361 : INFO : worker thread finished

...

2021-11-17 18:52:40,067 : INFO : deleting the raw counts dictionary of 20510 items
2021-11-17 18:52:40,070 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 18:52:40,074 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T18:52:40.074009', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 18:52:40,328 : INFO : estimated required memory for 20510 words and 100 dimensions: 29904800 bytes
2021-11-17 18:52:40,329 : INFO : resetting layer weights
2021-11-17 18:52:40,359 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 20510 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T18:52:40.359104', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

Model: pv-dbow, dm=0, dm_concat=0; ngrams=4, window=3
Corpus: TaggedDocument(['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7'], [0, 0])


2021-11-17 18:53:03,598 : INFO : deleting the raw counts dictionary of 20510 items
2021-11-17 18:53:03,601 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 18:53:03,602 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T18:53:03.602863', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 18:53:03,858 : INFO : estimated required memory for 20510 words and 100 dimensions: 29904800 bytes
2021-11-17 18:53:03,861 : INFO : resetting layer weights
2021-11-17 18:53:03,888 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 20510 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T18:53:03.888301', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

Model: pv-dbow, dm=0, dm_concat=0; ngrams=4, window=4
Corpus: TaggedDocument(['CM7-Eb7-G#M7-B7', 'Eb7-G#M7-B7-EM7', 'G#M7-B7-EM7-G7', 'B7-EM7-G7-Gm7', 'EM7-G7-Gm7-C7', 'G7-Gm7-C7-FM7', 'Gm7-C7-FM7-G#7', 'C7-FM7-G#7-C#M7', 'FM7-G#7-C#M7-E7', 'G#7-C#M7-E7-Am7', 'C#M7-E7-Am7-D7', 'E7-Am7-D7-Dm7', 'Am7-D7-Dm7-G7'], [0, 0])


2021-11-17 18:53:26,330 : INFO : deleting the raw counts dictionary of 20510 items
2021-11-17 18:53:26,333 : INFO : sample=0 downsamples 0 most-common words
2021-11-17 18:53:26,336 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 66584 word corpus (100.0%% of prior 66584)', 'datetime': '2021-11-17T18:53:26.336469', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-17 18:53:26,595 : INFO : estimated required memory for 20510 words and 100 dimensions: 29904800 bytes
2021-11-17 18:53:26,596 : INFO : resetting layer weights
2021-11-17 18:53:26,624 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 20510 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-17T18:53:26.624815', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GC

In [132]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}


for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    if not "npy" in name: ## !!! concat_x_y.model.syn1neg.npy cannot be loaded
      models[name] = Doc2Vec.load(file_name)

2021-11-17 18:53:48,297 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_4_3.model
2021-11-17 18:53:48,316 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_4_3.model.dv.* with mmap=None
2021-11-17 18:53:48,322 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_4_3.model.wv.* with mmap=None
2021-11-17 18:53:48,324 : INFO : setting ignored attribute cum_table to None


['autos_d2v_pv-dbow_4_3.model', 'autos_d2v_pv-dbow_2_4.model', 'autos_d2v_pv-dbow_1_4.model', 'autos_d2v_pv-dm_1_4.model', 'autos_d2v_pv-dbow_1_3.model', 'autos_d2v_pv-dbow_3_3.model', 'autos_d2v_pv-dm_3_4.model', 'autos_d2v_pv-dm_4_3.model', 'autos_d2v_pv-dm_3_3.model', 'autos_d2v_pv-dm_4_2.model', 'autos_d2v_pv-dbow_4_4.model', 'autos_d2v_pv-dm_3_2.model', 'autos_d2v_pv-dm_1_2.model', 'autos_d2v_pv-dbow_3_2.model', 'autos_d2v_pv-dm_2_2.model', 'autos_d2v_pv-dm_2_3.model', 'autos_d2v_pv-dbow_2_3.model', 'autos_d2v_pv-dbow_1_2.model', 'autos_d2v_pv-dbow_4_2.model', 'autos_d2v_pv-dm_1_3.model', 'autos_d2v_pv-dbow_2_2.model', 'autos_d2v_pv-dm_4_4.model', 'autos_d2v_pv-dm_2_4.model', 'autos_d2v_pv-dbow_3_4.model']
./models_doc2vec/autos_d2v_pv-dbow_4_3.model


2021-11-17 18:53:48,660 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_4_3.model', 'datetime': '2021-11-17T18:53:48.660803', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 18:53:48,666 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model
2021-11-17 18:53:48,679 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model.dv.* with mmap=None
2021-11-17 18:53:48,680 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_4.model.wv.* with mmap=None
2021-11-17 18:53:48,685 : INFO : setting ignored attribute cum_table to None
2021-11-17 18:53:48,734 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_2_4.model', 'datetime': '2021-11-17T18:53:48.734068', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

./models_doc2vec/autos_d2v_pv-dbow_2_4.model
./models_doc2vec/autos_d2v_pv-dbow_1_4.model
./models_doc2vec/autos_d2v_pv-dm_1_4.model
./models_doc2vec/autos_d2v_pv-dbow_1_3.model
./models_doc2vec/autos_d2v_pv-dbow_3_3.model


2021-11-17 18:53:48,996 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_3.model', 'datetime': '2021-11-17T18:53:48.996686', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 18:53:49,001 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_3_4.model
2021-11-17 18:53:49,019 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_3_4.model.dv.* with mmap=None
2021-11-17 18:53:49,024 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_3_4.model.wv.* with mmap=None
2021-11-17 18:53:49,025 : INFO : setting ignored attribute cum_table to None
2021-11-17 18:53:49,160 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_3_4.model', 'datetime': '2021-11-17T18:53:49.160414', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platf

...

2021-11-17 18:53:50,867 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_4_2.model.dv.* with mmap=None
2021-11-17 18:53:50,869 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_4_2.model.wv.* with mmap=None
2021-11-17 18:53:50,871 : INFO : setting ignored attribute cum_table to None
2021-11-17 18:53:51,120 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_4_2.model', 'datetime': '2021-11-17T18:53:51.120756', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 18:53:51,127 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_1_3.model
2021-11-17 18:53:51,135 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_3.model.dv.* with mmap=None
2021-11-17 18:53:51,137 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_3.model.wv.* with mmap=

./models_doc2vec/autos_d2v_pv-dm_1_3.model
./models_doc2vec/autos_d2v_pv-dbow_2_2.model
./models_doc2vec/autos_d2v_pv-dm_4_4.model


2021-11-17 18:53:51,481 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_4_4.model', 'datetime': '2021-11-17T18:53:51.480961', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-17 18:53:51,483 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_2_4.model
2021-11-17 18:53:51,492 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_4.model.dv.* with mmap=None
2021-11-17 18:53:51,494 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_4.model.wv.* with mmap=None
2021-11-17 18:53:51,496 : INFO : setting ignored attribute cum_table to None
2021-11-17 18:53:51,543 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_2_4.model', 'datetime': '2021-11-17T18:53:51.543572', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platfor

./models_doc2vec/autos_d2v_pv-dm_2_4.model
./models_doc2vec/autos_d2v_pv-dbow_3_4.model


2021-11-17 18:53:51,710 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4.model', 'datetime': '2021-11-17T18:53:51.710018', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [133]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dm_1_4.train', 'autos_d2v_pv-dm_1_3.train', 'autos_d2v_pv-dbow_3_4.train', 'autos_d2v_pv-dm_1_2.train', 'autos_d2v_pv-dbow_3_2.train', 'autos_d2v_pv-dbow_4_3.train', 'autos_d2v_pv-dbow_2_2.train', 'autos_d2v_pv-dm_2_4.train', 'autos_d2v_pv-dbow_2_3.train', 'autos_d2v_pv-dbow_2_4.train', 'autos_d2v_pv-dbow_1_4.train', 'autos_d2v_pv-dm_3_3.train', 'autos_d2v_pv-dm_4_2.train', 'autos_d2v_pv-dbow_4_2.train', 'autos_d2v_pv-dm_3_4.train', 'autos_d2v_pv-dbow_3_3.train', 'autos_d2v_pv-dm_3_2.train', 'autos_d2v_pv-dm_4_4.train', 'autos_d2v_pv-dbow_4_4.train', 'autos_d2v_pv-dbow_1_3.train', 'autos_d2v_pv-dm_2_2.train', 'autos_d2v_pv-dm_2_3.train', 'autos_d2v_pv-dbow_1_2.train', 'autos_d2v_pv-dm_4_3.train']
./traindata_doc2vec/autos_d2v_pv-dm_1_4.train
./traindata_doc2vec/autos_d2v_pv-dm_1_3.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_4.train
./traindata_doc2vec/autos_d2v_pv-dm_1_2.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_2.train
./traindata_doc2vec/autos_d2v_pv-dbow_4_3.tra

#### Determine Similarity of Tunes

In [134]:
#assert(len(model.dv.vectors == len(list(title_to_sectionid.keys()))))

In [136]:
tunes_eval_list = [
  'These Foolish Things', 
]

In [137]:
title_to_sectionid["These Foolish Things"]

[3866, 3867, 3868, 3869]

In [162]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      for model_name, model in models.items():
          print(f"{model_name}")
          sim = model.dv.most_similar(section)
          #print(f"'{sectionid_to_section[section]}' is most similar to:")
          for index, value in sim:
              print(f'\t{value:.3f}: {sectionid_to_section[index]}')

              # don't store results to the own tune
              #if index not in title_to_sectionid[tune]:
              if True:
                  df_sim.loc[len(df_sim)] = [tune, 
                                            model_name, 
                                            sectionid_to_section[section], 
                                            sectionid_to_section[index], 
                                            f"{value:.3f}",
                                             sectionid_to_title[index]]



These Foolish Things

*** These Foolish Things, section1 (A) ***
autos_d2v_pv-dbow_4_3.model
	0.956: These Foolish Things, section2 (A)
	0.942: These Foolish Things, section4 (A)
	0.906: Emily, section2 (B)
	0.903: I'm Old Fashioned, section3 (C)
	0.890: Tangerine, section2 (B)
	0.890: Touch Of Your Lips, The, section3 (A)
	0.886: People, section3 (A)
	0.885: Touch Of Your Lips, The, section1 (A)
	0.883: People, section1 (A)
	0.879: Why Do I Love You, section2 (A)
autos_d2v_pv-dbow_2_4.model
	0.929: These Foolish Things, section4 (A)
	0.902: These Foolish Things, section2 (A)
	0.869: Put On A Happy Face, section2 (B)
	0.832: Nearness Of You, The, section3 (B)
	0.785: Emily, section2 (B)
	0.771: Way You Look Tonight, The, section1 (A)
	0.767: Stairway To The Stars, section1 (A)
	0.766: Street Of Dreams, section2 (A)
	0.762: Blue Room, The, section1 (A)
	0.762: Why Do I Love You, section2 (A)
autos_d2v_pv-dbow_1_4.model
	0.918: These Foolish Things, section4 (A)
	0.866: Nearness Of You, 

In [139]:
from google.colab import files

file = 'doc2vec_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [140]:
df_sim.sort_values(['sim_value'], ascending=False).head(50)

,title1,model,ref_section,sim_section,sim_value,sim_title
621,These Foolish Things,autos_d2v_pv-dm_4_4.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.984,Isn't It A Pity
411,These Foolish Things,autos_d2v_pv-dbow_4_3.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.984,Isn't It A Pity
501,These Foolish Things,autos_d2v_pv-dm_4_2.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.983,Isn't It A Pity
511,These Foolish Things,autos_d2v_pv-dbow_4_4.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.981,Isn't It A Pity
591,These Foolish Things,autos_d2v_pv-dbow_4_2.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.980,Isn't It A Pity
481,These Foolish Things,autos_d2v_pv-dm_4_3.model,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.978,Isn't It A Pity
298,These Foolish Things,autos_d2v_pv-dm_3_2.model,"These Foolish Things, section2 (A)","You're Everything, section1 (A)",0.951,You're Everything
718,These Foolish Things,autos_d2v_pv-dm_3_3.model,"These Foolish Things, section4 (A)","Nardis, section3 (B)",0.945,Nardis
76,These Foolish Things,autos_d2v_pv-dm_4_2.model,"These Foolish Things, section1 (A)","Emily, section2 (B)",0.944,Emily
59,These Foolish Things,autos_d2v_pv-dm_4_3.model,"These Foolish Things, section1 (A)","Emily, section2 (B)",0.942,Emily


In [141]:
df_sim.loc[:, 'model'].unique()

array(['autos_d2v_pv-dbow_4_3.model', 'autos_d2v_pv-dbow_2_4.model',
       'autos_d2v_pv-dbow_1_4.model', 'autos_d2v_pv-dm_1_4.model',
       'autos_d2v_pv-dbow_1_3.model', 'autos_d2v_pv-dbow_3_3.model',
       'autos_d2v_pv-dm_3_4.model', 'autos_d2v_pv-dm_4_3.model',
       'autos_d2v_pv-dm_3_3.model', 'autos_d2v_pv-dm_4_2.model',
       'autos_d2v_pv-dbow_4_4.model', 'autos_d2v_pv-dm_3_2.model',
       'autos_d2v_pv-dm_1_2.model', 'autos_d2v_pv-dbow_3_2.model',
       'autos_d2v_pv-dm_2_2.model', 'autos_d2v_pv-dm_2_3.model',
       'autos_d2v_pv-dbow_2_3.model', 'autos_d2v_pv-dbow_1_2.model',
       'autos_d2v_pv-dbow_4_2.model', 'autos_d2v_pv-dm_1_3.model',
       'autos_d2v_pv-dbow_2_2.model', 'autos_d2v_pv-dm_4_4.model',
       'autos_d2v_pv-dm_2_4.model', 'autos_d2v_pv-dbow_3_4.model'],
      dtype=object)

In [152]:
dd = df_sim.query(f'model == "autos_d2v_pv-dm_4_4.model"').sort_values('sim_value', ascending=False).head(50)

In [157]:
dd.value_counts('sim_title')

sim_title
We See                               3
Emily                                3
It's Only a Paper Moon               3
Old Folks                            2
Old Cape Cod                         2
Ablution                             1
All Of Me                            1
As Time Goes By                      1
Blue Lou                             1
Born To Be Blue                      1
Bud Powell                           1
Cry Me A River                       1
Dinah                                1
Eiderdown                            1
Elora                                1
For Sentimental Reasons              1
Feel Like Makin' Love                1
Tiny Capers                          1
I Left My Heart In San Francisco     1
Isn't It A Pity                      1
It's The Talk Of The Town            1
More Than You Know                   1
Morning                              1
Smoke Gets In Your Eyes              1
Stairway To The Stars                1
Them There Eyes

In [167]:
for model_name in sorted(models):
  if "dbow_" in model_name:
    print(f"\n*** {model_name} ***")
    dd = df_sim.query(f'model == "{model_name}"').sort_values('sim_value', ascending=False).head(50)
    dd_count = dd.value_counts('sim_title')
    print(dd_count)


*** autos_d2v_pv-dbow_1_2.model ***
sim_title
These Foolish Things                 5
It's Easy To Remember                4
I Loves You Porgy                    2
Taking A Chance On Love              2
Isn't It A Pity                      2
You Took Advantage Of Me             1
Let's Get Lost                       1
Blueberry Hill                       1
But Beautiful                        1
Come Back To Me                      1
Emily                                1
I've Told Eve'ry Little Star         1
If I Had You                         1
If There Is Someone Lovelier         1
Imagination                          1
Just Friends                         1
Manhattan                            1
Where Are You                        1
More Than You Know                   1
Nearness Of You, The                 1
Put On A Happy Face                  1
Rosetta                              1
Stairway To The Stars                1
Teach Me Tonight                     1
They Can't Take T

Try out a model on multiple tunes

In [177]:
best_model = 'autos_d2v_pv-dbow_3_4.model'
model = models[best_model]

In [178]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [179]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      sim = model.dv.most_similar(section)
      #print(f"'{sectionid_to_section[section]}' is most similar to:")
      for index, value in sim:
          print(f'\t{value:.3f}: {sectionid_to_section[index]}')

          # don't store results to the own tune
          #if index not in title_to_sectionid[tune]:
          if True:
              df_sim.loc[len(df_sim)] = [tune, 
                                        model_name, 
                                        sectionid_to_section[section], 
                                        sectionid_to_section[index], 
                                        f"{value:.3f}",
                                          sectionid_to_title[index]]



Sweet Sue, Just You

*** Sweet Sue, Just You, section1 (A) ***
	0.998: Sweet Sue, Just You, section2 (A)
	0.998: Sweet Sue, Just You, section4 (A)
	0.953: Honeysuckle Rose, section2 (A)
	0.913: Honeysuckle Rose, section1 (A)
	0.911: Seven Come Eleven, section2 (A)
	0.909: Honeysuckle Rose, section4 (A)
	0.908: Seven Come Eleven, section4 (A)
	0.902: Seven Come Eleven, section1 (A)
	0.892: We See, section3 (B)
	0.874: What A Little Moonlight Can Do, section1 (A)

*** Sweet Sue, Just You, section2 (A) ***
	0.999: Sweet Sue, Just You, section4 (A)
	0.998: Sweet Sue, Just You, section1 (A)
	0.953: Honeysuckle Rose, section2 (A)
	0.915: Seven Come Eleven, section2 (A)
	0.913: Honeysuckle Rose, section1 (A)
	0.912: Seven Come Eleven, section4 (A)
	0.908: Honeysuckle Rose, section4 (A)
	0.905: Seven Come Eleven, section1 (A)
	0.895: We See, section3 (B)
	0.873: Wow, section2 (B)

*** Sweet Sue, Just You, section3 (B) ***
	0.951: Sweet Sue, section3 (B)
	0.903: You're My Thrill, section3 (B)
	

# Good Matches

* Misty - Portrait Of Jennie
* Misty - September in the Rain
* Misty - I May Be Wrong


